# Project: Predicting Stock Price & Return  
## 2. Extrat Transform Load (ETL)
Data source include: Alphavantage free API, Bank of Canada, Google Trends

In this notebook, I clean the data that were collected from the first notebook and put them all together in a single csv file.

Ver. 1.0  
Remove unecessary content  

Ver. 0.1  
The ETL pipeline, including interpolating and cleaning

## Table of Content  
[0. Import packages](#0)  
[1. Extract Data](#1)  
[2. Transform the Data](#2)  
[3. Load the Data](#3)  

<a class="anchor" id="0"></a>
#### 0. Import packages

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
#let the notebook display full length of the data columns
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

<a class="anchor" id="1"></a>
#### 1. Extract Data

##### Define extract functions  
These functions can be easily changed if we want to change the data acquisition methods.

In [3]:
def extract_stock_data(name):
    '''
    read stock data from csv file
    '''
    data = pd.read_csv("data/"+ name + ".csv", parse_dates=['date'], index_col='date')
    return data

In [4]:
def extract_indecis_data(name):
    '''
    read index data from csv file
    '''
    data = pd.read_csv("data/"+name + ".csv", parse_dates=['date'], index_col='date')
    return data

In [5]:
def extract_exchange_data(name):
    '''
    read exchange data from csv file
    '''
    data = pd.read_csv("data/"+name + ".csv", parse_dates=['date'], index_col='date')
    return data

In [6]:
def extract_trend_data(name):
    '''
    read trend data from csv file
    '''
    data = pd.read_csv("data/"+"trend_" + name + ".csv", parse_dates=['date'], index_col='date')
    return data

In [7]:
def extract_data(stock_list=['loblaw', 'metro', 'empa', 'gwl', 'atd', 'tsx', 'sp500'],
                 indecis_list=['BCPI', 'CPI', 'bank_interest'], 
                 exchange_list=['CEER'], 
                 trend_list=['grocery_store', 'loblaws', 'stock']):

    data = pd.DataFrame()
    for stock in stock_list:
        stock_data = extract_stock_data(stock)
        stock_data.columns = [stock + '_price', stock + '_volume']
        data = data.join(stock_data, how='outer')

    for index in indecis_list:
        indecis_data = extract_indecis_data(index)
        indecis_data.columns = [index]
        data = data.join(indecis_data, how='outer')

    for exchange in exchange_list:
        exchange_data = extract_indecis_data(exchange)
        exchange_data.columns = [exchange]
        data = data.join(exchange_data, how='outer')

    for trend in trend_list:
        trend_data = extract_trend_data(trend)
        trend_data.columns = ["trend_" + trend]
        data = data.join(trend_data, how='outer')

    return data


##### Extract the data using the functions  
Inspect the data and we will see lots of missing values. This is because the data have different starting and ending date, and some data have lower frequency such as weekly and monthly.

In [8]:
data_extracted = extract_data(stock_list=['loblaw', 'metro', 'empa', 'gwl', 'atd', 'tsx', 'sp500'],
                              indecis_list=['BCPI', 'CPI', 'bank_interest'], 
                              exchange_list=['CEER'], 
                              trend_list=['grocery_store', 'loblaws', 'stock'])

In [9]:
data_extracted.head()

,loblaw_price,loblaw_volume,metro_price,metro_volume,empa_price,empa_volume,gwl_price,gwl_volume,atd_price,atd_volume,tsx_price,tsx_volume,sp500_price,sp500_volume,BCPI,CPI,bank_interest,CEER,trend_grocery_store,trend_loblaws,trend_stock
date,,,,,,,,,,,,,,,,,,,,,
1999-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.43,NaN,NaN,NaN
1999-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.47,NaN,NaN,NaN
1999-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.84,NaN,NaN,NaN
1999-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,96.66,NaN,NaN,NaN
1999-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,96.46,NaN,NaN,NaN


In [10]:
data_extracted.tail()

,loblaw_price,loblaw_volume,metro_price,metro_volume,empa_price,empa_volume,gwl_price,gwl_volume,atd_price,atd_volume,tsx_price,tsx_volume,sp500_price,sp500_volume,BCPI,CPI,bank_interest,CEER,trend_grocery_store,trend_loblaws,trend_stock
date,,,,,,,,,,,,,,,,,,,,,
2019-10-21,71.42,455500.0,55.9270,320200.0,34.92,491852.0,109.16,99100.0,39.27,748298.0,16418.5000,155105700.0,3006.72,3.271620e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-10-22,70.77,613100.0,55.6879,247800.0,35.02,469645.0,108.15,104800.0,39.03,1083171.0,16391.5000,171979500.0,2995.99,3.523890e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-10-23,69.85,595400.0,55.1000,516000.0,34.45,467906.0,106.70,106000.0,39.53,1228126.0,16335.9004,181906700.0,3004.52,3.392870e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-10-24,69.22,785600.0,54.9500,296500.0,34.46,368114.0,106.01,80300.0,39.08,1151289.0,16369.2998,167159100.0,3010.29,3.692600e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-10-25,69.60,1068100.0,54.9200,338100.0,34.75,352352.0,106.06,73000.0,39.81,1262531.0,16404.5000,173518300.0,3022.55,3.370370e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
data_extracted.to_csv("data/data_extracted.csv")

In [12]:
pd.read_csv("data/data_extracted.csv").head()

,date,loblaw_price,loblaw_volume,metro_price,metro_volume,empa_price,empa_volume,gwl_price,gwl_volume,atd_price,atd_volume,tsx_price,tsx_volume,sp500_price,sp500_volume,BCPI,CPI,bank_interest,CEER,trend_grocery_store,trend_loblaws,trend_stock
0,1999-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.43,NaN,NaN,NaN
1,1999-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.47,NaN,NaN,NaN
2,1999-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.84,NaN,NaN,NaN
3,1999-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,96.66,NaN,NaN,NaN
4,1999-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,96.46,NaN,NaN,NaN


In [13]:
data_extracted.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5500 entries, 1999-01-01 to 2019-10-25
Data columns (total 21 columns):
loblaw_price           5024 non-null float64
loblaw_volume          5024 non-null float64
metro_price            5025 non-null float64
metro_volume           5025 non-null float64
empa_price             5025 non-null float64
empa_volume            5025 non-null float64
gwl_price              5024 non-null float64
gwl_volume             5024 non-null float64
atd_price              5025 non-null float64
atd_volume             5025 non-null float64
tsx_price              5025 non-null float64
tsx_volume             5025 non-null float64
sp500_price            4986 non-null float64
sp500_volume           4986 non-null float64
BCPI                   1030 non-null float64
CPI                    237 non-null float64
bank_interest          1030 non-null float64
CEER                   5421 non-null float64
trend_grocery_store    190 non-null float64
trend_loblaws         

In [14]:
data_extracted.describe()

,loblaw_price,loblaw_volume,metro_price,metro_volume,empa_price,empa_volume,gwl_price,gwl_volume,atd_price,atd_volume,tsx_price,tsx_volume,sp500_price,sp500_volume,BCPI,CPI,bank_interest,CEER,trend_grocery_store,trend_loblaws,trend_stock
count,5024.000000,5.024000e+03,5025.000000,5.025000e+03,5025.000000,5.025000e+03,5024.000000,5.024000e+03,5025.000000,5.025000e+03,5025.000000,5.025000e+03,4986.000000,4.986000e+03,1030.000000,237.000000,1030.000000,5421.000000,190.000000,190.000000,190.000000
mean,35.614063,5.294897e+05,16.226961,7.020978e+05,13.328838,2.283551e+05,71.211612,1.156642e+05,9.980139,2.031178e+06,11839.875345,1.720095e+08,1560.950916,3.087643e+09,474.326641,115.605907,4.006359,119.529555,51.421053,38.878947,26.221053
std,12.903062,5.919713e+05,15.044590,8.047871e+05,8.344615,2.973981e+05,21.421331,1.033845e+05,12.293850,2.336902e+06,2969.689206,7.572380e+07,569.467685,1.478108e+09,133.807475,11.783501,1.397609,13.244301,15.005169,8.555519,20.481081
min,0.000000,0.000000e+00,1.097700,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.244500,0.000000e+00,0.000000,0.000000e+00,676.530000,3.560700e+08,222.700000,93.500000,2.250000,95.430000,34.000000,24.000000,11.000000
25%,26.301225,2.512000e+05,4.049200,3.462000e+05,5.512100,6.075000e+04,56.003950,5.840000e+04,1.526800,9.532980e+05,9278.500000,1.238915e+08,1149.667575,1.662200e+09,365.982500,105.400000,3.000000,110.300000,41.250000,35.000000,14.000000
50%,32.526400,4.144500e+05,8.960600,5.450000e+05,11.501400,1.480480e+05,68.726950,9.200000e+04,2.750200,1.560404e+06,12268.299800,1.719795e+08,1356.700000,3.212905e+09,451.600000,115.600000,3.700000,118.330000,46.000000,38.000000,17.000000
75%,44.075475,6.445000e+05,23.148700,8.538000e+05,20.089000,3.024660e+05,88.391075,1.393000e+05,17.614800,2.368018e+06,14209.599600,2.133231e+08,1969.280000,3.917825e+09,580.987500,125.700000,4.750000,131.310000,60.000000,41.000000,26.750000
max,75.770000,1.482520e+07,58.308400,3.157200e+07,37.233700,5.316156e+06,118.568400,1.635000e+06,43.794600,6.585227e+07,16899.699200,8.588881e+08,3025.860100,1.145623e+10,911.940000,137.000000,7.500000,150.480000,100.000000,100.000000,100.000000


<a class="anchor" id="2"></a>
    
#### 2. Transform the data

In [15]:
def transform_data(data_extracted,
                              stock_list=['loblaw', 'metro', 'empa', 'gwl', 'atd', 'tsx', 'sp500'],
                              indecis_list=['BCPI', 'CPI', 'bank_interest'], 
                              exchange_list=['CEER'], 
                              trend_list=['grocery_store', 'loblaws', 'stock']):
    '''
    tranform the data into a usable form. include slicing the date, interpolating values, etc.
    '''
    data_extracted = data_extracted.resample('B').interpolate()

    data_extracted = data_extracted['2003':]

    data_extracted = data_extracted[1:]

    for column in [column for column in data_extracted.columns if ('price' in column) or ('volume' in column)]:
        for row in data_extracted[data_extracted[column] == 0].iterrows():
            '''
            last_price = data_extracted.loc[:row[0]].tail(2).head(1)
            while last_price[column].squeeze() == 0:
                last_price = data_extracted.loc[:last_price.index.max()].tail(2).head(1)
            last_price = last_price[column].squeeze()

            next_price = data_extracted.loc[row[0]:].head(2).tail(1)
            while next_price[column].squeeze() == 0:
                next_price = data_extracted.loc[next_price.index.min():].head(2).tail(1)
            next_price = next_price[column].squeeze()

            data_extracted.loc[row[0]][column] = np.mean([last_price, next_price])
            '''
            # The previous solution was to use mean value to subsitute the 0 value of price, now we decide to drop these rows
            data_extracted.drop(row[0], inplace=True)

    return data_extracted


In [16]:
data_transformed = transform_data(data_extracted)

In [17]:
data_extracted.describe()

,loblaw_price,loblaw_volume,metro_price,metro_volume,empa_price,empa_volume,gwl_price,gwl_volume,atd_price,atd_volume,tsx_price,tsx_volume,sp500_price,sp500_volume,BCPI,CPI,bank_interest,CEER,trend_grocery_store,trend_loblaws,trend_stock
count,5024.000000,5.024000e+03,5025.000000,5.025000e+03,5025.000000,5.025000e+03,5024.000000,5.024000e+03,5025.000000,5.025000e+03,5025.000000,5.025000e+03,4986.000000,4.986000e+03,1030.000000,237.000000,1030.000000,5421.000000,190.000000,190.000000,190.000000
mean,35.614063,5.294897e+05,16.226961,7.020978e+05,13.328838,2.283551e+05,71.211612,1.156642e+05,9.980139,2.031178e+06,11839.875345,1.720095e+08,1560.950916,3.087643e+09,474.326641,115.605907,4.006359,119.529555,51.421053,38.878947,26.221053
std,12.903062,5.919713e+05,15.044590,8.047871e+05,8.344615,2.973981e+05,21.421331,1.033845e+05,12.293850,2.336902e+06,2969.689206,7.572380e+07,569.467685,1.478108e+09,133.807475,11.783501,1.397609,13.244301,15.005169,8.555519,20.481081
min,0.000000,0.000000e+00,1.097700,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.244500,0.000000e+00,0.000000,0.000000e+00,676.530000,3.560700e+08,222.700000,93.500000,2.250000,95.430000,34.000000,24.000000,11.000000
25%,26.301225,2.512000e+05,4.049200,3.462000e+05,5.512100,6.075000e+04,56.003950,5.840000e+04,1.526800,9.532980e+05,9278.500000,1.238915e+08,1149.667575,1.662200e+09,365.982500,105.400000,3.000000,110.300000,41.250000,35.000000,14.000000
50%,32.526400,4.144500e+05,8.960600,5.450000e+05,11.501400,1.480480e+05,68.726950,9.200000e+04,2.750200,1.560404e+06,12268.299800,1.719795e+08,1356.700000,3.212905e+09,451.600000,115.600000,3.700000,118.330000,46.000000,38.000000,17.000000
75%,44.075475,6.445000e+05,23.148700,8.538000e+05,20.089000,3.024660e+05,88.391075,1.393000e+05,17.614800,2.368018e+06,14209.599600,2.133231e+08,1969.280000,3.917825e+09,580.987500,125.700000,4.750000,131.310000,60.000000,41.000000,26.750000
max,75.770000,1.482520e+07,58.308400,3.157200e+07,37.233700,5.316156e+06,118.568400,1.635000e+06,43.794600,6.585227e+07,16899.699200,8.588881e+08,3025.860100,1.145623e+10,911.940000,137.000000,7.500000,150.480000,100.000000,100.000000,100.000000


In [18]:
data_transformed.describe()

,loblaw_price,loblaw_volume,metro_price,metro_volume,empa_price,empa_volume,gwl_price,gwl_volume,atd_price,atd_volume,tsx_price,tsx_volume,sp500_price,sp500_volume,BCPI,CPI,bank_interest,CEER,trend_grocery_store,trend_loblaws,trend_stock
count,4361.000000,4.361000e+03,4361.000000,4.361000e+03,4361.000000,4.361000e+03,4361.000000,4.361000e+03,4361.000000,4.361000e+03,4361.000000,4.361000e+03,4361.000000,4.361000e+03,4361.000000,4361.000000,4361.000000,4361.000000,4109.000000,4109.000000,4109.000000
mean,37.147310,5.695757e+05,18.913607,7.194956e+05,15.287601,2.482847e+05,73.409281,1.209610e+05,11.813733,2.196316e+06,12595.170847,1.860470e+08,1625.598647,3.407853e+09,506.350460,118.998076,3.682573,124.282160,51.306065,38.841198,26.564944
std,13.070197,5.789205e+05,15.011572,8.258242e+05,7.716913,2.840124e+05,20.817372,1.039773e+05,12.626393,2.226329e+06,2532.315722,7.119453e+07,589.046743,1.348808e+09,119.141573,9.753498,1.123584,10.066851,14.789069,8.275570,20.758822
min,16.836000,2.450000e+04,2.975300,1.140000e+04,3.808200,2.385000e+03,33.383500,5.000000e+03,0.575400,6.559800e+04,6228.600100,1.824000e+05,676.530000,4.872200e+08,277.380000,102.006250,2.250000,96.700000,34.000000,24.000000,11.000000
25%,26.488900,2.929000e+05,6.696700,3.725000e+05,8.647200,7.899000e+04,56.680200,6.380000e+04,2.167900,1.179834e+06,11424.599600,1.466113e+08,1182.769950,2.567540e+09,407.790000,111.609302,3.000000,116.290000,41.325581,35.581395,14.050000
50%,34.187500,4.555000e+05,12.066650,5.631000e+05,14.231800,1.770210e+05,70.010100,9.620000e+04,3.753600,1.680192e+06,12816.000000,1.844234e+08,1400.500000,3.400350e+09,498.164000,119.872727,3.000000,123.320000,46.000000,38.023256,16.619048
75%,48.546100,6.820000e+05,33.383600,8.597000e+05,21.268500,3.265560e+05,93.301100,1.443000e+05,25.938600,2.493000e+06,14595.649900,2.212560e+08,2061.720000,4.065230e+09,604.926000,127.004651,4.250000,133.830000,60.069767,40.625000,26.043478
max,75.770000,1.482520e+07,58.308400,3.157200e+07,37.233700,4.790751e+06,118.568400,1.635000e+06,43.794600,6.585227e+07,16899.699200,8.588881e+08,3025.860100,1.145623e+10,911.940000,137.000000,6.250000,150.480000,100.000000,100.000000,100.000000


In [19]:
data_transformed.head()

,loblaw_price,loblaw_volume,metro_price,metro_volume,empa_price,empa_volume,gwl_price,gwl_volume,atd_price,atd_volume,tsx_price,tsx_volume,sp500_price,sp500_volume,BCPI,CPI,bank_interest,CEER,trend_grocery_store,trend_loblaws,trend_stock
date,,,,,,,,,,,,,,,,,,,,,
2003-01-02,31.6037,134900.0,3.1212,101400.0,5.1538,17982.0,65.7853,133000.0,0.7302,296784.0,6740.1001,76406700.0,909.03,1.229200e+09,305.898,102.00625,4.5,96.70,NaN,NaN,NaN
2003-01-03,31.3404,48500.0,3.1387,273600.0,5.1797,34761.0,65.7062,9300.0,0.7091,2904300.0,6772.7002,74780400.0,908.59,1.130800e+09,306.836,102.01250,4.5,97.08,NaN,NaN,NaN
2003-01-06,31.5452,232700.0,3.1247,161700.0,5.1971,64512.0,65.8931,99400.0,0.7036,541584.0,6837.2998,142266300.0,929.01,1.435900e+09,307.774,102.01875,4.5,97.34,NaN,NaN,NaN
2003-01-07,31.6037,842600.0,3.1387,945900.0,5.2317,366657.0,66.1089,54400.0,0.7004,685632.0,6802.7998,150351200.0,922.93,1.545200e+09,308.712,102.02500,4.5,97.42,NaN,NaN,NaN
2003-01-08,31.5452,423900.0,3.1212,301800.0,5.1624,179112.0,65.8212,227400.0,0.6759,1697196.0,6723.1001,145587400.0,909.93,1.467600e+09,309.650,102.03125,4.5,97.21,NaN,NaN,NaN


In [20]:
data_transformed.tail()

,loblaw_price,loblaw_volume,metro_price,metro_volume,empa_price,empa_volume,gwl_price,gwl_volume,atd_price,atd_volume,tsx_price,tsx_volume,sp500_price,sp500_volume,BCPI,CPI,bank_interest,CEER,trend_grocery_store,trend_loblaws,trend_stock
date,,,,,,,,,,,,,,,,,,,,,
2019-10-21,71.42,455500.0,55.9270,320200.0,34.92,491852.0,109.16,99100.0,39.27,748298.0,16418.5000,155105700.0,3006.72,3.271620e+09,432.33,136.8,3.95,117.75,80.0,34.0,64.0
2019-10-22,70.77,613100.0,55.6879,247800.0,35.02,469645.0,108.15,104800.0,39.03,1083171.0,16391.5000,171979500.0,2995.99,3.523890e+09,432.33,136.8,3.95,117.75,80.0,34.0,64.0
2019-10-23,69.85,595400.0,55.1000,516000.0,34.45,467906.0,106.70,106000.0,39.53,1228126.0,16335.9004,181906700.0,3004.52,3.392870e+09,432.33,136.8,3.95,117.75,80.0,34.0,64.0
2019-10-24,69.22,785600.0,54.9500,296500.0,34.46,368114.0,106.01,80300.0,39.08,1151289.0,16369.2998,167159100.0,3010.29,3.692600e+09,432.33,136.8,3.95,117.75,80.0,34.0,64.0
2019-10-25,69.60,1068100.0,54.9200,338100.0,34.75,352352.0,106.06,73000.0,39.81,1262531.0,16404.5000,173518300.0,3022.55,3.370370e+09,432.33,136.8,3.95,117.75,80.0,34.0,64.0


In [21]:
data_transformed.to_csv('data/data_transformed.csv')

<a class="anchor" id="3"></a>

#### 3. Load the Data

In [22]:
def load_data(data_transformed_filename):
    '''
    save the transformed data to csv file
    '''
    data_loaded = pd.read_csv(data_transformed_filename, parse_dates=['date'], index_col='date')

    return data_loaded

In [23]:
data_loaded = load_data("data/data_transformed.csv")

In [24]:
data_loaded.head()

,loblaw_price,loblaw_volume,metro_price,metro_volume,empa_price,empa_volume,gwl_price,gwl_volume,atd_price,atd_volume,tsx_price,tsx_volume,sp500_price,sp500_volume,BCPI,CPI,bank_interest,CEER,trend_grocery_store,trend_loblaws,trend_stock
date,,,,,,,,,,,,,,,,,,,,,
2003-01-02,31.6037,134900.0,3.1212,101400.0,5.1538,17982.0,65.7853,133000.0,0.7302,296784.0,6740.1001,76406700.0,909.03,1.229200e+09,305.898,102.00625,4.5,96.70,NaN,NaN,NaN
2003-01-03,31.3404,48500.0,3.1387,273600.0,5.1797,34761.0,65.7062,9300.0,0.7091,2904300.0,6772.7002,74780400.0,908.59,1.130800e+09,306.836,102.01250,4.5,97.08,NaN,NaN,NaN
2003-01-06,31.5452,232700.0,3.1247,161700.0,5.1971,64512.0,65.8931,99400.0,0.7036,541584.0,6837.2998,142266300.0,929.01,1.435900e+09,307.774,102.01875,4.5,97.34,NaN,NaN,NaN
2003-01-07,31.6037,842600.0,3.1387,945900.0,5.2317,366657.0,66.1089,54400.0,0.7004,685632.0,6802.7998,150351200.0,922.93,1.545200e+09,308.712,102.02500,4.5,97.42,NaN,NaN,NaN
2003-01-08,31.5452,423900.0,3.1212,301800.0,5.1624,179112.0,65.8212,227400.0,0.6759,1697196.0,6723.1001,145587400.0,909.93,1.467600e+09,309.650,102.03125,4.5,97.21,NaN,NaN,NaN


In [25]:
data_loaded.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4361 entries, 2003-01-02 to 2019-10-25
Data columns (total 21 columns):
loblaw_price           4361 non-null float64
loblaw_volume          4361 non-null float64
metro_price            4361 non-null float64
metro_volume           4361 non-null float64
empa_price             4361 non-null float64
empa_volume            4361 non-null float64
gwl_price              4361 non-null float64
gwl_volume             4361 non-null float64
atd_price              4361 non-null float64
atd_volume             4361 non-null float64
tsx_price              4361 non-null float64
tsx_volume             4361 non-null float64
sp500_price            4361 non-null float64
sp500_volume           4361 non-null float64
BCPI                   4361 non-null float64
CPI                    4361 non-null float64
bank_interest          4361 non-null float64
CEER                   4361 non-null float64
trend_grocery_store    4109 non-null float64
trend_loblaws       

In [26]:
data_loaded.describe()

,loblaw_price,loblaw_volume,metro_price,metro_volume,empa_price,empa_volume,gwl_price,gwl_volume,atd_price,atd_volume,tsx_price,tsx_volume,sp500_price,sp500_volume,BCPI,CPI,bank_interest,CEER,trend_grocery_store,trend_loblaws,trend_stock
count,4361.000000,4.361000e+03,4361.000000,4.361000e+03,4361.000000,4.361000e+03,4361.000000,4.361000e+03,4361.000000,4.361000e+03,4361.000000,4.361000e+03,4361.000000,4.361000e+03,4361.000000,4361.000000,4361.000000,4361.000000,4109.000000,4109.000000,4109.000000
mean,37.147310,5.695757e+05,18.913607,7.194956e+05,15.287601,2.482847e+05,73.409281,1.209610e+05,11.813733,2.196316e+06,12595.170847,1.860470e+08,1625.598647,3.407853e+09,506.350460,118.998076,3.682573,124.282160,51.306065,38.841198,26.564944
std,13.070197,5.789205e+05,15.011572,8.258242e+05,7.716913,2.840124e+05,20.817372,1.039773e+05,12.626393,2.226329e+06,2532.315722,7.119453e+07,589.046743,1.348808e+09,119.141573,9.753498,1.123584,10.066851,14.789069,8.275570,20.758822
min,16.836000,2.450000e+04,2.975300,1.140000e+04,3.808200,2.385000e+03,33.383500,5.000000e+03,0.575400,6.559800e+04,6228.600100,1.824000e+05,676.530000,4.872200e+08,277.380000,102.006250,2.250000,96.700000,34.000000,24.000000,11.000000
25%,26.488900,2.929000e+05,6.696700,3.725000e+05,8.647200,7.899000e+04,56.680200,6.380000e+04,2.167900,1.179834e+06,11424.599600,1.466113e+08,1182.769950,2.567540e+09,407.790000,111.609302,3.000000,116.290000,41.325581,35.581395,14.050000
50%,34.187500,4.555000e+05,12.066650,5.631000e+05,14.231800,1.770210e+05,70.010100,9.620000e+04,3.753600,1.680192e+06,12816.000000,1.844234e+08,1400.500000,3.400350e+09,498.164000,119.872727,3.000000,123.320000,46.000000,38.023256,16.619048
75%,48.546100,6.820000e+05,33.383600,8.597000e+05,21.268500,3.265560e+05,93.301100,1.443000e+05,25.938600,2.493000e+06,14595.649900,2.212560e+08,2061.720000,4.065230e+09,604.926000,127.004651,4.250000,133.830000,60.069767,40.625000,26.043478
max,75.770000,1.482520e+07,58.308400,3.157200e+07,37.233700,4.790751e+06,118.568400,1.635000e+06,43.794600,6.585227e+07,16899.699200,8.588881e+08,3025.860100,1.145623e+10,911.940000,137.000000,6.250000,150.480000,100.000000,100.000000,100.000000
